# Logistic Regression with Grid Search (scikit-learn)

<a href="https://colab.research.google.com/github/VertaAI/modeldb-client/blob/master/workflows/demos/sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# restart your notebook if prompted on Colab
try:
    import verta
except ModuleNotFoundError:
    !pip install verta

This example features:
- **scikit-learn**'s `LinearRegression` model
- **scikit-learn**'s `GridSearchCV` utility for performing grid search and cross-validation
- **verta**'s Python client logging the grid search results
- **verta**'s Python client retrieving the best run from the grid search to calculate full training accuracy
- predictions against a deployed model

In [ ]:
HOST = "demo.app.verta.ai"

PROJECT_NAME = "Census Income Classification"
EXPERIMENT_NAME = "Logistic Regression"

In [ ]:
#import os
#os.environ['VERTA_EMAIL'] = 
#os.environ['VERTA_DEV_KEY'] = 

## Imports

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

import itertools
import time
from multiprocessing import Pool

import six

import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

In [ ]:
try:
    import wget
except ModuleNotFoundError:
    !pip install wget  # you may need pip3
    import wget

---

# Log Workflow

## Prepare Data

In [ ]:
train_data_url = "http://s3.amazonaws.com/verta-starter/census-train.csv"
train_data_filename = wget.download(train_data_url)
test_data_url = "http://s3.amazonaws.com/verta-starter/census-test.csv"
test_data_filename = wget.download(test_data_url)

In [ ]:
df_train = pd.read_csv(train_data_filename)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:, -1]

df_train.head()

## Prepare Hyperparameters

In [ ]:
hyperparam_candidates = {
    'C': [1e-6, 1e-4],
    'solver': ['lbfgs'],
    'max_iter': [15, 28],
}
hyperparam_sets = [dict(zip(hyperparam_candidates.keys(), values))
                   for values
                   in itertools.product(*hyperparam_candidates.values())]

## Instantiate Client

In [ ]:
from verta import Client
from verta.utils import ModelAPI

client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

## Run Validation

In [ ]:
def run_experiment(hyperparams):
    # create object to track experiment run
    run = client.set_experiment_run()
    
    # create validation split
    (X_val_train, X_val_test,
     y_val_train, y_val_test) = model_selection.train_test_split(X_train, y_train,
                                                                 test_size=0.2,
                                                                 shuffle=True)

    # log hyperparameters
    run.log_hyperparameters(hyperparams)
    print(hyperparams, end=' ')
    
    # create and train model
    model = linear_model.LogisticRegression(**hyperparams)
    model.fit(X_train, y_train)
    
    # calculate and log validation accuracy
    val_acc = model.score(X_val_test, y_val_test)
    run.log_metric("val_acc", val_acc)
    print("Validation accuracy: {:.4f}".format(val_acc))
    
    # create deployment artifacts
    model_api = ModelAPI(X_train, y_train)
    requirements = six.StringIO('\n'.join(["scikit-learn==0.21.1",
                                           "cloudpickle==1.1.1"]))
    
    # save and log model
    run.log_model_for_deployment(model, model_api, requirements, X_train, y_train)
    
with Pool() as pool:
    pool.map(run_experiment, hyperparam_sets)

---

# Revisit Workflow

## Retrieve Best Run

In [ ]:
best_run = expt.expt_runs.sort("metrics.val_acc", descending=True)[0]
print("Validation Accuracy: {:.4f}".format(best_run.get_metric("val_acc")))

best_hyperparams = best_run.get_hyperparameters()
print("Hyperparameters: {}".format(best_hyperparams))

## Train on Full Dataset

In [ ]:
model = linear_model.LogisticRegression(**best_hyperparams, multi_class='auto')
model.fit(X_train, y_train)

## Calculate Accuracy on Full Training Set

In [ ]:
train_acc = model.score(X_train, y_train)
print("Training accuracy: {:.4f}".format(train_acc))

---

# Make Live Predictions

In [ ]:
model_id = 

## Prepare Data

In [ ]:
df_test = pd.read_csv(test_data_filename)
X_test = df_test.iloc[:,:-1]

## Load Deployed Model

In [ ]:
from verta._demo_utils import DeployedModel

deployed_model = DeployedModel(HOST, model_id)

## Query Deployed Model

In [ ]:
for x in itertools.cycle(X_test.values.tolist()):
    print(deployed_model.predict([x]))
    time.sleep(.5)

---